In [ ]:
!pip install torch==2.8.0+cu126 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [2]:
!pip install transformers[torch] langchain langchain_community langgraph langchain_huggingface

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
    ----------------

# Основной рабочий пайплайн работы с агентом

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage
from langchain_core.prompts import PromptTemplate

In [ ]:
#Qwen/Qwen2.5-0.5B-Instruct
model_id = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
pipe = pipeline(
    "text-generation",
    'Qwen/Qwen2.5-3B-Instruct',
    temperature = 0.001,
    max_new_tokens=1000,
    device=0,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
hf_pipeline = HuggingFacePipeline(pipeline=pipe)
chat_model = ChatHuggingFace(llm=hf_pipeline,)

In [4]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

In [ ]:
class Subtopic(BaseModel):
  title: str = Field(..., description = 'Technocal title of subtopic')
  description: str = Field(..., description = '1-2 sentence explantion')

class SubtopicList(BaseModel):
  subtopics: List[Subtopic]


parser =PydanticOutputParser(pydantic_object=SubtopicList)

In [ ]:
prompt_template = PromptTemplate(
    template=(
    '''
{format_insrt}\n
You're a machine learning expert.\n
There's a topic: {topic}\n
Compose the titles of 3 articles that fit this topic.\n
Please answer in Russian.\n
Output JSON object: {{{{"subtopics": [{{{{"title": "...", "description": "..."}}}}, ...]}}}}\n
    '''),
    input_variables=['topic'],
    partial_variables={'format_insrt': parser.get_format_instructions()},
)

In [5]:
from langchain_core.runnables.base import Runnable
from typing import Optional, Any
from langchain_core.runnables import RunnableConfig

class TrimResponseRunnable(Runnable):
    def invoke(self, input: Any, config: Optional[RunnableConfig] = None, **kwargs: Any) -> Any:
        text = input
        if hasattr(text, "content"):
            text.content = text.content
        start_pos = text.content.find('<|im_start|>assistant')
        if start_pos != -1:
            text.content = text.content[start_pos+len('<|im_start|>assistant'):]
        return text
trim_response = TrimResponseRunnable()

In [ ]:
chain = prompt_template | chat_model | trim_response | parser

In [ ]:
responce = chain.invoke({'topic':'Машинное обучение в России'})

In [ ]:
print(responce.content)

<|im_start|>assistant
```json
{
    "subtopics": [
        {
            "title": "Машинное обучение в России: Начальные принципы",
            "description": "Этот раздел рассказывает о основных принципах и методах обучения машинного intelligence в России. Содержит информацию о различных подходах к обучению машинного обучения, алгоритмах, системам обучения и их применении."
        },
        {
            "title": "Машинное обучение в России: Современные технологии",
            "description": "Этот раздел описывает современные технологии в области машинного обучения, такие как искусственный интеллект, нейронные сети и др. В этом разделе будет рассмотрено использование таких технологий в различных сферах жизни и бизнеса."
        },
        {
            "title": "Машинное обучение в России: Проблемы и возможности",
            "description": "Этот раздел обсуждает проблемы и возможности применения машинного обучения в России. Он включает в себя анализ проблем, связанных с обучением 

## Использование инструментов

In [6]:
from langchain.tools import tool
import requests
import json

@tool
def get_weather(input: str) -> str:
    """Get the current weather for the specified city.

    Args:
        input: The name of the city in Russian (for example: 'Москва', 'Санкт-Петербург')

    Returns:
        A line with information about temperature and weather conditions
    """
    try:
        # Демо-версия без реального API
        weather_data = {
            "Москва": "🌤️ +15°C, облачно с прояснениями",
            "Санкт-Петербург": "🌧️ +12°C, небольшой дождь",
            "Сочи": "☀️ +22°C, солнечно",
            "Новосибирск": "❄️ -5°C, снег"
        }
        return weather_data.get(input, f"Погода для {input} временно недоступна")
    except Exception as e:
        return f"Ошибка: {str(e)}"

@tool
def calculate(input: str) -> str:
    """Calculate a mathematical expression.

    Args:
        input: A mathematical expression (for example: '2 + 2 * 3', ' sin(45)')

    Returns:
        Calculation result
    """
    try:
        # Безопасное вычисление
        allowed_chars = set('0123456789+-*/.() ')
        if all(c in allowed_chars for c in input):
            result = eval(input)
            return f"Результат: {input} = {result}"
        else:
            return "Ошибка: выражение содержит недопустимые символы"
    except Exception as e:
        return f"Ошибка вычисления: {str(e)}"

@tool
def search_information(input: str) -> str:
    """Search for information on a given query.

    Args:
        input: A search query in Russian

    Returns:
        Information found
    """
    # Демо-версия поиска
    knowledge_base = {
        "столица россии": "Столица России - Москва",
        "самая длинная река": "Самая длинная река в России - Лена (4400 км)",
        "население москвы": "Население Москвы около 13 миллионов человек"
    }

    query_lower = input.lower()
    for key, value in knowledge_base.items():
        if key in query_lower:
            return value

    return f"По запросу '{input}' информация не найдена в базе знаний"

# Создаем список инструментов
tools = [get_weather, calculate, search_information]

In [ ]:
def format_tools_description(tools_list):
    descriptions = []
    for tool in tools_list:
        desc = f"""{tool.name}:
   - Description: {tool.description}
   - Input data: {', '.join([f'{param_name}: {param_type}' for param_name, param_type in tool.args.items()])}"""
        descriptions.append(desc)
    return "\n\n".join(descriptions)

tools_description = format_tools_description(tools)


prompt_template = PromptTemplate(
    template=(
'''
You are an intelligent assistant with access to the following tools:

AVAILABLE TOOLS:
{tools_description}


INSTRUCTIONS FOR USE:
  1. Carefully analyze the user's request
  2. Determine if you need a response tool.
  3. Use ONLY the tools from the list of AVAILABLE TOOLS.
  4. If you NEED a tool to respond, start the response with the [TOOL] tag and then place the JSON object.
  5. If you DON'T NEED the tool, reply in plain text.

examples:

Query: "Какая погода в Москве?"
Response: [TOOL] {{"tool": "get_weather", "input": "Москва"}}

Request: "Привет! Как дела?"
Answer: Привет! Я ИИ ассистент. Чем могу помочь?

STRICT RULES:
- DON'T FORGET that YOU can communicate with the user and keep the conversation going.
- NEVER write explanations before or after JSON
- NEVER use keys other than "tool", "input"
- NEVER come up with new tool names - use ONLY AVAILABLE TOOLS from the list.
- There must be a [TOOL] BEFORE the JSON.
- ANSWER AND VALUE IN RUSSIAN



USER'S QUESTION: {input}

'''),
    input_variables=['input'],
    partial_variables={'tools_description': tools_description},
)

In [8]:
ask_result_quest = PromptTemplate(
    input_variables=["user_question", "previous_result"],
    template=(
        "Был вопрос пользователя:\n{user_question}\n"
        "Из инструмента ты выяснил, что:\n{previous_result}\n"
        "Проанализируй результат работы инструмента и дай ответ пользователю ПОНЯТНО БЕЗ ЛИШНЕГО."
        "НИЧЕГО НЕ придумывай и НЕ дополняй. Работай с тем что есть."
    )
)


In [15]:
chain = prompt_template | chat_model | trim_response | process

In [16]:
resp = {'input':'Когда родился Александр Пушкин?'}
responce = chain.invoke(resp)

In [17]:
responce.content

'\nИзвините, но информация о том, когда родился Александр Пушкин, не найдена в базе знаний.'

In [132]:
process.invoke(responce)

AIMessage(content="По запросу 'Когда родился Александр Пушкин' информация не найдена в базе знаний", additional_kwargs={}, response_metadata={}, id='run--bd33b3b9-25c9-4fb3-874a-d030485a6177-0')

In [9]:
from langchain import LLMChain
def get_result(responce):
  # Создаем цепочку для нового шага
  ask_chain = ask_result_quest | chat_model | trim_response

  # Вызов с переданными переменными
  result = ask_chain.invoke({
      "user_question": resp['input'],
      "previous_result": responce
  })
  return result.content


In [137]:
print(result.content)


Информация о дате рождения Александра Пушкина не найдена в базе знаний.


In [ ]:
responce.content = responce.content.replace("query", "input")

In [10]:
from typing import Literal, Optional
from pydantic import BaseModel, ValidationError

class ToolCall(BaseModel):
    tool: Literal["get_weather", "search_information", "calculate"]  # только разрешенные инструменты
    input: str
    # добавьте другие необходимые параметры

In [11]:
def is_valid_input(tool_name: str, input_data: str) -> bool:
    """Проверяет корректность входных данных для конкретного инструмента"""

    validators = {
        "get_weather": lambda x: len(x.strip()) > 0 and x.isprintable(),
        "search_information": lambda x: len(x.strip()) >= 2,
        "calculate": lambda x: is_valid_expression(x)
    }

    validator = validators.get(tool_name)
    return validator(input_data) if validator else False

def is_valid_expression(expr: str) -> bool:
    """Проверяет безопасность математического выражения"""
    # Запрещаем опасные конструкции
    dangerous_keywords = ['import', 'exec', 'eval', '__', 'open', 'file']
    return all(keyword not in expr for keyword in dangerous_keywords)

In [ ]:
import json
import logging

def parse_ai_response(response_text: str) -> Optional[ToolCall]:
    """
    Безопасно парсит ответ ИИ и валидирует его
    """
    try:
        # Пытаемся распарсить JSON
        cleaned_response = response_text.strip()

        # Убираем возможные markdown блоки кода
        if cleaned_response.startswith('```json'):
            cleaned_response = cleaned_response[7:]  # убираем ```json
        if cleaned_response.endswith('```'):
            cleaned_response = cleaned_response[:-3]  # убираем ```
        cleaned_response = cleaned_response.strip()

        data = json.loads(cleaned_response)

        # Валидируем через Pydantic
        tool_call = ToolCall(**data)

        # Дополнительные проверки бизнес-логики
        if not is_valid_input(tool_call.tool, tool_call.input):
            logging.warning(f"Invalid input for tool {tool_call.tool}: {tool_call.input}")
            return None

        return tool_call

    except json.JSONDecodeError as e:
        logging.error(f"Invalid JSON from AI: {e}")
        return None
    except ValidationError as e:
        logging.error(f"Validation error: {e}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None

In [13]:
class ProcessAIResponse(Runnable):
    def invoke(self, input: Any, config: Optional[RunnableConfig] = None, **kwargs: Any) -> Any:
        text = input
        if hasattr(text, "content"):
            text.content = text.content
        text.content = process_ai_response(text.content)
        return text
process = ProcessAIResponse()

In [14]:
def process_ai_response(ai_response: str):
    """
    Основной процесс обработки ответа ИИ
    """
    # Парсим и валидируем
    if '[TOOL]' in ai_response:
      start = ai_response.find('[TOOL]')
      ask = ai_response[start+7:].strip()
      tool_call = parse_ai_response(ask)

      if not tool_call:
          # Обработка ошибки
          return {"error": "Invalid AI response"}

      # Выполняем соответствующий инструмент
      try:
          result = execute_tool(tool_call.tool, tool_call.input)
          result = get_result(result)
          return result
      except Exception as e:
          logging.error(f"Tool execution failed: {e}")
          return {"error": f"Tool execution failed: {str(e)}"}

    else:
      return ai_response

def execute_tool(tool_name: str, input_data: str):
    """Выполняет конкретный инструмент"""

    tools = {
        "get_weather": get_weather,
        "search_information": search_information,
        "calculate": calculate
    }

    if tool_name not in tools:
        raise ValueError(f"Unknown tool: {tool_name}")

    return tools[tool_name].invoke(input_data)

In [ ]:
process_ai_response(responce.content)

{'success': True, 'result': 'Результат: 6 + 12 * 43 - 222 = 300'}

In [ ]:
responce.content[len('[TOOL]')+1:].strip()

'{"tool": "get_weather", "input": "Санкт-Петербург"}'

## Память

In [18]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Создаем память
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history",
    input_key="input"
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2736\3041127763.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [19]:
def chat_with_memory():
    while True:
        user_input = input("Вы: ")
        if user_input.lower() == 'выход':
            break
        resp['input'] = user_input
        # Получаем историю из памяти
        history = memory.load_memory_variables({})["history"]
        history_text = "\n".join([f"{msg.type}: {msg.content}" for msg in history])

        # Генерируем ответ
        response = chain.invoke({
            "input": user_input,
            "history": history_text
        })

        print(f"Ассистент: {response.content}")

        # Сохраняем в память
        memory.save_context({"input": user_input}, {"output": response.content})

In [ ]:
chat_with_memory()

## Задачи

* [ ] Построить граф состояний и определить случай, когда должна быть обратная связь
* [ ] Разобраться с памятью и попробовать ее применить
* [ ] Переходить от примера к конкретным инструментам по диплому